In [1]:
import os
import json

psat_json_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'PSAT/Data/2023_PSAT.json'))
psat_json = json.load(open(psat_json_path))

In [2]:
psat_json[33]

{'id': '2023_PSAT_34',
 'paragraph': '부서에서 검토 중인 과제를 여섯 개의 범주, ‘중점 추진 과제’, ‘타 부서와 협의가 필요한 과제’, ‘많은 예산이 필요한 과제’, ‘장기 시행 과제’, ‘인력 재배치가 필요한 과제’, ‘즉각적인 효과가 나타나는 과제’로 나누어 검토해 본 결과는 다음과 같다.○중점 추진 과제 가운데 인력 재배치가 필요한 과제는 없지만 장기 시행 과제는 있다.○타 부서와 협의가 필요한 과제 가운데 즉각적인 효과가 나타나는 과제는 없다.○많은 예산이 필요한 과제 가운데 즉각적인 효과가 나타나는 과제가 있다.○장기 시행 과제 가운데 타 부서와 협의가 필요하지 않은 과제는 모두 인력 재배치가 필요한 과제이다.○인력 재배치가 필요한 과제 가운데 많은 예산이 필요한 과제는 없다.',
 'problems': [{'question': '34. 다음 글의 내용이 참일 때 반드시 참인 것만을 에서 모두 고르면 ?',
   'choices': ['① ㄱ', '② ㄷ', '③ ㄱ, ㄴ', '④ ㄴ, ㄷ', '⑤ ㄱ, ㄴ, ㄷ'],
   'additional': 'ㄱ.장기 시행 과제이면서 즉각적인 효과가 나타나는 과제 가운데는 많은 예산이 필요한 과제가 없다.ㄴ.인력 재배치가 필요하지 않은 과제 가운데 즉각적인 효과가 나타나지 않는 과제가 있다.ㄷ.장기 시행 과제가 아니면서 많은 예산이 필요한 과제가 있다.'}]}

In [3]:
basic_prompt = """Let’s think step by step.
다음 지문을 읽고, 질문에 대한 답을 1부터 5까지 중에 한 개만 골라서 대답하시오.
지문: {}
질문: {}
선택지: 1번: {}, 2번: {}, 3번: {}, 4번: {}, 5번: {}
1번, 2번, 3번, 4번, 5번 중에서 한 개의 답을 고르시오. 정답:
"""
    
basic_additional_prompt = """Let’s think step by step.
다음 지문을 읽고, 질문에 대한 답을 1부터 5까지 중에 한 개만 골라서 대답하시오.
지문: {}
질문: {}
보기: {}
선택지: 1번: {}, 2번: {}, 3번: {}, 4번: {}, 5번: {}
1번, 2번, 3번, 4번, 5번 중에서 한 개의 답을 고르시오. 정답:
"""

In [4]:
def get_prompt(question, additional=False):
    # Assuming there's only one problem per paragraph for simplification
    problem = question['problems'][0]
    
    if 'additional' in problem.keys():
        additional = True
    
    if additional:
        return basic_additional_prompt.format(question['paragraph'], 
                                              problem['question'], 
                                              problem['additional'], 
                                              problem['choices'][0], 
                                              problem['choices'][1],
                                              problem['choices'][2],
                                              problem['choices'][3],
                                              problem['choices'][4])
    else:
        return basic_prompt.format(question['paragraph'], 
                                   problem['question'], 
                                   problem['choices'][0], 
                                   problem['choices'][1],
                                   problem['choices'][2],
                                   problem['choices'][3],
                                   problem['choices'][4])

In [5]:
basic_prompt_input = [get_prompt(question) for question in psat_json]

In [11]:
print(basic_prompt_input[27])

Let’s think step by step.
다음 지문을 읽고, 질문에 대한 답을 1부터 5까지 중에 한 개만 골라서 대답하시오.
지문: 앎을 추구하는 사람이라면, 어떤 명제 P가 거짓인 경우에 그 명제를 믿지 않아야 한다. 이 직관을 설명하기 위해 민감성 조건이 제시되었다. 이에 따르면, 명제 P에 대한 믿음이 ‘민감하다’면, P가 거짓인 가상의 경우에는 P를 믿지 않아야 한다. 민감성 조건을 옹호하는 철학자는, 명제 “지구는 자전한다.”를 우리가 안다고 할 수 있는 이유는 그 명제가 참일 뿐만 아니라 민감성 조건을 충족하기 때문이라고 생각한다. 즉, “지구는 자전한다.”가 거짓인 가상의 경우에 우리는 그 명제를 믿지 않을 것이기 때문이라는 것이다.  일견 그럴듯해 보이는 민감성 조건의 문제점은 다음과 같은 상황을 고려할 때 잘 드러난다. 철이는 지금 자신의 손자인 민수가 마당에서 건강하게 뛰어놀고 있는 모습을 직접 지켜보면서 “민수가 건강하다.”라는 명제 Q를 믿고 있다고 하자. 철이가 Q를 안다고 하려면, Q에 대한 철이의 믿음은 민감성 조건을 충족해야 한다. 하지만 다음과 같은 가상의 경우를 상상해보자. 민수가 크게 다쳐서 병원에 입원했는데, 철이가 매우 신뢰하는 자신의 아들이 “민수가 친구 집에서 건강하게 놀고 있다.”라고 철이에게 알려줬다. 이 경우, 철이는 Q를 믿을 것이다. 따라서 Q에 대한 철이의 믿음은 민감성 조건을 충족하지 못하는 것이다.  그러나 지금 마당에서 건강하게 뛰어놀고 있는 모습을 직접 지켜보는 철이가 Q를 알지 못한다고 말하는 것은 분명 잘못이다. 따라서, “ ”라고 결론지을 수 있다.
질문: 28. 다음 글의 빈칸에 들어갈 내용으로 가장 적절한 것은?
선택지: 1번: ① 어떤 경우에서도 참인 명제만이 앎의 대상일 수 있다., 2번: ② 어떤 명제가 앎의 대상이라고 해서 그 명제에 대한 믿음이 민감할 필요는 없다., 3번: ③ 어떤 명제에 대한 믿음이 민감하다는 것은 그 명제를 알기 위한 충분 조건이 아니다., 4번: ④ 믿음

In [ ]:
instruct_prompt = """Let's think step by step.
다음 지문을 읽고, 질문에 대한 답을 1부터 5까지 중에 한 개만 골라서 대답하시오.
지문: {}
질문: {}
선택지: 1번: {}, 2번: {}, 3번: {}, 4번: {}, 5번: {}
유의사항:
지문을 읽을 때 지문의 모든 내용을 정확히 파악하고 분석해야 한다. 
지문의 주장이나 근거를 대충 넘기지 말고 각각의 문장을 세심하게 살펴봐야 한다.
문제를 풀기 전, 지문의 소재와 주장이 어떤 도메인에 해당하는 내용인지 정리해야 한다.
문제를 풀 때, 지문에서 명시되어 있는 직접적인 근거를 중심으로 푸는 것이 우선이다.
하지만 지문으로만 추론이 되지 않는 경우에는 기본적인 상식을 바탕으로 간접적인 추론해서 문제를 풀어야 한다. 
그럼에도 지문의 내용을 기반으로 너무 복잡한 추론이나 가정을 하는 것은 위험하므로 최소한의 상식을 가지고 추론한다.
마지막으로 선택한 답안이 지문과 보기의 내용에 잘 맞는지 다시 한 번 점검한다.
유의사항의 지시를 따라 1번, 2번, 3번, 4번, 5번 중에서 한 개의 답을 고르시오. 정답:
"""
    
basic_instruct_prompt = """Let's think step by step.
다음 지문을 읽고, 질문에 대한 답을 1부터 5까지 중에 한 개만 골라서 대답하시오.
지문: {}
질문: {}
보기: {}
선택지: 1번: {}, 2번: {}, 3번: {}, 4번: {}, 5번: {}
유의사항:
지문을 읽을 때 지문의 모든 내용을 정확히 파악하고 분석해야 한다. 
지문의 주장이나 근거를 대충 넘기지 말고 각각의 문장을 세심하게 살펴봐야 한다.
문제를 풀기 전, 지문의 소재와 주장이 어떤 도메인에 해당하는 내용인지 정리해야 한다.
문제를 풀 때, 지문에서 명시되어 있는 직접적인 근거를 중심으로 푸는 것이 우선이다.
하지만 지문으로만 추론이 되지 않는 경우에는 기본적인 상식을 바탕으로 간접적인 추론해서 문제를 풀어야 한다. 
그럼에도 지문의 내용을 기반으로 너무 복잡한 추론이나 가정을 하는 것은 위험하므로 최소한의 상식을 가지고 추론한다.
마지막으로 선택한 답안이 지문과 보기의 내용에 잘 맞는지 다시 한 번 점검한다.
유의사항의 지시를 따라 1번, 2번, 3번, 4번, 5번 중에서 한 개의 답을 고르시오. 정답:
"""